# Bloch Simulation - Reproducible

**Mode**: Re-run simulation from parameters

This notebook reproduces the simulation from scratch using the exported parameters.

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from bloch_simulator import (
    BlochSimulator, TissueParameters,
    SpinEcho, SpinEchoTipAxis, GradientEcho,
    SliceSelectRephase, design_rf_pulse
)

# Set matplotlib style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Simulation Parameters

In [ ]:
# Define simulation parameters

# Tissue parameters
tissue_name = 'White Matter'
t1 = 0.830000  # seconds
t2 = 0.070000  # seconds
density = 1.000

# Sequence parameters
sequence_type = 'Spin Echo'
te = 0.030000  # seconds
tr = 0.200000  # seconds
flip_angle = 90.0  # degrees

# Simulation parameters
num_positions = 3
num_frequencies = 5
time_step_us = 1.000
mode = 2 if 'time-resolved' == 'time-resolved' else 0


## Initialize Simulator

In [ ]:
# Create simulator
use_parallel = False
num_threads = 4

sim = BlochSimulator(use_parallel=use_parallel, num_threads=num_threads)

# Create tissue
tissue = TissueParameters(
    name=tissue_name,
    t1=t1,
    t2=t2,
    density=density
)

print(f"Simulator initialized")
print(f"  Tissue: {tissue.name}")
print(f"  T1: {tissue.t1*1000:.1f} ms, T2: {tissue.t2*1000:.1f} ms")


## Define Pulse Sequence

In [ ]:
# Create Spin Echo sequence
sequence = SpinEcho(
    te=te,
    tr=tr
)
print(f"Spin Echo sequence: TE={te*1000:.1f} ms, TR={tr*1000:.1f} ms")


## Spatial and Frequency Sampling

In [ ]:
# Define spatial positions
positions = np.zeros((num_positions, 3))
if num_positions > 1:
    positions[:, 2] = np.linspace(-0.010000, 0.010000, num_positions)

# Define off-resonance frequencies
if num_frequencies > 1:
    frequencies = np.linspace(-100.0, 100.0, num_frequencies)
else:
    frequencies = np.array([0.0])

print(f"Sampling:")
print(f"  Positions: {num_positions}")
print(f"  Frequencies: {num_frequencies}")


## Run Simulation

In [ ]:
# Run simulation
print("\nRunning simulation...")

result = sim.simulate(
    sequence,
    tissue,
    positions=positions,
    frequencies=frequencies,
    mode=mode
)

# Extract results for easier access
data = {
    'mx': result['mx'],
    'my': result['my'],
    'mz': result['mz'],
    'signal': result['signal'],
    'time': result['time'],
    'positions': result['positions'],
    'frequencies': result['frequencies'],
    'tissue': {'name': tissue.name, 't1': tissue.t1, 't2': tissue.t2}
}

print(f"Simulation complete!")
print(f"  Result shape: {result['mx'].shape}")
print(f"  Duration: {result['time'][-1]*1000:.3f} ms")


## Visualization

In [ ]:
# Plot magnetization evolution
position_idx = 0  # Change to plot different position
freq_idx = 0      # Change to plot different frequency

if data['mx'].ndim == 3:  # Time-resolved
    time_ms = data['time'] * 1000
    mx = data['mx'][:, position_idx, freq_idx]
    my = data['my'][:, position_idx, freq_idx]
    mz = data['mz'][:, position_idx, freq_idx]
    mxy = np.sqrt(mx**2 + my**2)

    fig, axes = plt.subplots(2, 2, figsize=(12, 8))

    axes[0, 0].plot(time_ms, mx, 'b-', linewidth=1.5)
    axes[0, 0].set_xlabel('Time (ms)')
    axes[0, 0].set_ylabel('Mx')
    axes[0, 0].set_title('Transverse Magnetization (x)')
    axes[0, 0].grid(True, alpha=0.3)

    axes[0, 1].plot(time_ms, my, 'r-', linewidth=1.5)
    axes[0, 1].set_xlabel('Time (ms)')
    axes[0, 1].set_ylabel('My')
    axes[0, 1].set_title('Transverse Magnetization (y)')
    axes[0, 1].grid(True, alpha=0.3)

    axes[1, 0].plot(time_ms, mz, 'g-', linewidth=1.5)
    axes[1, 0].set_xlabel('Time (ms)')
    axes[1, 0].set_ylabel('Mz')
    axes[1, 0].set_title('Longitudinal Magnetization')
    axes[1, 0].grid(True, alpha=0.3)

    axes[1, 1].plot(time_ms, mxy, color='purple', linewidth=1.5)
    axes[1, 1].set_xlabel('Time (ms)')
    axes[1, 1].set_ylabel('|Mxy|')
    axes[1, 1].set_title('Transverse Magnitude')
    axes[1, 1].grid(True, alpha=0.3)

    plt.suptitle(f'Magnetization Evolution - Position {position_idx}, Frequency {freq_idx}',
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("Endpoint data - no time evolution to plot")


## Signal Analysis

In [ ]:
# Plot signal
if data['signal'].ndim == 3:  # Time-resolved
    signal = data['signal'][:, position_idx, freq_idx]
    time_ms = data['time'] * 1000

    fig, axes = plt.subplots(2, 1, figsize=(12, 8))

    axes[0].plot(time_ms, np.real(signal), 'b-', label='Real', linewidth=1.5)
    axes[0].plot(time_ms, np.imag(signal), 'r-', label='Imaginary', linewidth=1.5)
    axes[0].set_xlabel('Time (ms)')
    axes[0].set_ylabel('Signal')
    axes[0].set_title('Complex Signal Components')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    axes[1].plot(time_ms, np.abs(signal), color='purple', linewidth=1.5)
    axes[1].set_xlabel('Time (ms)')
    axes[1].set_ylabel('|Signal|')
    axes[1].set_title('Signal Magnitude')
    axes[1].grid(True, alpha=0.3)

    plt.suptitle(f'MRI Signal - Position {position_idx}, Frequency {freq_idx}',
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("Endpoint data - no time evolution to plot")


## Save Results (Optional)

In [ ]:
# Uncomment to save results
# sim.save_results('simulation_results.h5', sequence_params, simulation_params)
# print('Results saved!')